In [0]:
%pip install --quiet --upgrade scikit-learn==1.4.2 tensorflow==2.16.1 scikeras==0.13.0 joblib



In [0]:
dbutils.library.restartPython()

In [0]:
import os

# Altere esta linha para o caminho que você criou no Passo 3
MODEL_DIR = "/Volumes/transacoes_db/copper/files/pix_fraud/production_artifacts/" 

print(f"Verificando o diretório local: {os.path.abspath(MODEL_DIR)}")

try:
    files = os.listdir(MODEL_DIR)
    if not files:
        print("Pasta vazia. Você colocou os ficheiros .pkl aqui?")
    else:
        print("Ficheiros encontrados:")
        for f in files:
            print(f"- {f}")
except FileNotFoundError:
    print(f"ERRO: A pasta '{MODEL_DIR}' não foi encontrada. Verifique o caminho.")
except NotADirectoryError:
    print(f"ERRO: O caminho '{MODEL_DIR}' não é um diretório/pasta.")

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import mlflow
import joblib
import json
import os
import warnings
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, recall_score, precision_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping
from mlflow.models.signature import infer_signature
from datetime import datetime

# --- Constantes de Produção ---
# Este deve ser o mesmo diretório do Volume usado no treinamento
MODEL_DIR = "/Volumes/transacoes_db/copper/files/pix_fraud/production_artifacts/"
BINARY_THRESHOLD = 0.5 # Threshold de decisão para fraude binária

# Suprimir warnings de serialização/versão que podem ocorrer
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suprimir logs do TensorFlow

# --- Definição da Função de Engenharia de Features ---
# (Necessário para carregar o pipeline)

def feature_engineer_datetimes(df_in: pd.DataFrame) -> pd.DataFrame:
    """
    Recebe um DataFrame com as colunas de data/hora e retorna
    um DataFrame com features numéricas de engenharia.
    """
    df = df_in.copy()
    df_out = pd.DataFrame(index=df.index)
    now = datetime.now()
    dt_tx = pd.to_datetime(df['data_transacao'])
    df_out['tx_hora_do_dia'] = dt_tx.dt.hour
    df_out['tx_dia_da_semana'] = dt_tx.dt.dayofweek
    df_out['tx_mes'] = dt_tx.dt.month
    
    # <--- Esta linha está comentada para corresponder ao pipeline treinado
    # df_out['pagador_idade_conta_dias'] = (now - pd.to_datetime(df['pagador_conta_aberta_em'])).dt.days
    
    df_out['recebedor_idade_conta_dias'] = (now - pd.to_datetime(df['recebedor_conta_aberta_em'])).dt.days
    df_out['pagador_idade_anos'] = ((now - pd.to_datetime(df['pagador_data_nascimento'])).dt.days / 365.25).astype(int)
    df_out['recebedor_idade_anos'] = ((now - pd.to_datetime(df['recebedor_data_nascimento'])).dt.days / 365.25).astype(int)
    df_out = df_out.fillna(0)
    return df_out


# --- Definições das Funções de Construção de Modelo ---
# (Necessário para carregar os KerasClassifiers)

def build_binary_model(meta):
    """Constrói o modelo binário para o KerasClassifier."""
    n_features_in = meta["n_features_in_"]
    tf.random.set_seed(42) 
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(n_features_in,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        loss='binary_crossentropy', 
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
        metrics=['AUC', tf.keras.metrics.Recall(name='recall'), tf.keras.metrics.Precision(name='precision')]
    )
    return model

def build_multiclass_model(meta):
    """Constrói o modelo multiclasse para o KerasClassifier."""
    n_features_in = meta["n_features_in_"]
    n_classes_out = meta["n_classes_"]
    tf.random.set_seed(42)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(n_features_in,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(n_classes_out, activation='softmax')
    ])
    model.compile(
        loss='sparse_categorical_crossentropy', 
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
        metrics=['accuracy']
    )
    return model


# --- Carregamento de Artefatos ---

# <--- MODIFICADO: Os nomes dos ficheiros foram corrigidos aqui ---
def carregar_artefatos(model_dir: str):
    """
    Carrega todos os artefatos necessários (pipelines e mapa de labels)
    do diretório especificado (no Volume).
    """
    print(f"Carregando artefatos de: {model_dir}")
    try:
        # Nomes atualizados para corresponder aos ficheiros reais na pasta
        pipeline_binario = joblib.load(os.path.join(model_dir, "fraud_binary_pipeline.pkl"))
        pipeline_multiclass = joblib.load(os.path.join(model_dir, "fraud_type_pipeline.pkl"))
        label_map_path = os.path.join(model_dir, "fraud_type_label_map.json")
            
        with open(label_map_path, 'r') as f:
            label_map_str_keys = json.load(f)
            # Converter chaves do JSON (string) de volta para inteiros
            label_map = {int(k): v for k, v in label_map_str_keys.items()}
            
        print("Artefatos (2 pipelines, 1 mapa de labels) carregados com sucesso.")
        return pipeline_binario, pipeline_multiclass, label_map

    except FileNotFoundError as e:
        print(f"Erro crítico: Arquivo de modelo/mapa não encontrado. Verifique os nomes dos arquivos. {e}")
        return None, None, None
    except Exception as e:
        print(f"Erro inesperado ao carregar artefatos: {e}")
        return None, None, None

# --- Função de Predição em Cascata (COM CORREÇÃO) ---

def prever_fraude(df_novo: pd.DataFrame, pipeline_binario, pipeline_multiclass, label_map) -> pd.DataFrame:
    """
    Executa a predição em cascata para novas transações.
    Recebe um DataFrame Pandas com o schema exato da view de features.
    """
    if pipeline_binario is None or pipeline_multiclass is None or label_map is None:
        raise ValueError("Modelos ou mapa de labels não foram carregados corretamente.")

    if not isinstance(df_novo, pd.DataFrame):
        df_novo = pd.DataFrame([df_novo])
        
    df_results = df_novo.copy()

    # 1. Modelo Binário
    try:
        fraud_probabilities = pipeline_binario.predict_proba(df_novo)[:, 1]
    except Exception as e:
        print(f"Erro na predição binária: {e}")
        df_results['predicted_fraud'] = np.nan
        df_results['fraud_probability'] = np.nan
        df_results['predicted_fraud_type'] = None
        df_results['predicted_type_probability'] = np.nan
        df_results['type_probabilities'] = None
        df_results['resumo_da_predicao'] = "Erro na predição" 
        return df_results

    df_results['fraud_probability'] = fraud_probabilities
    df_results['predicted_fraud'] = (fraud_probabilities >= BINARY_THRESHOLD).astype(int)

    # Inicializar colunas
    df_results['predicted_fraud_type'] = None
    df_results['predicted_type_probability'] = np.nan
    df_results['type_probabilities'] = [{} for _ in range(len(df_results))]

    # 2. Modelo Multiclasse (só para fraudes)
    indices_fraude = df_results[df_results['predicted_fraud'] == 1].index

    if not indices_fraude.empty:
        df_fraude = df_results.loc[indices_fraude]
        
        try:
            type_probs_matrix = pipeline_multiclass.predict_proba(df_fraude)

            # --- INÍCIO DA CORREÇÃO ---
            # O 'predict_proba' para uma única amostra retorna um array 1D (ex: [0.1, 0.8, 0.1]).
            # As funções np.argmax(axis=1) e np.max(axis=1) exigem um array 2D.
            # Esta verificação garante que 'type_probs_matrix' seja sempre 2D.
            if type_probs_matrix.ndim == 1:
                # Transforma [0.1, 0.8, 0.1] em [[0.1, 0.8, 0.1]]
                type_probs_matrix = np.array([type_probs_matrix])
            # --- FIM DA CORREÇÃO ---

            # Agora estas linhas funcionarão com 1 ou N amostras
            type_predictions_indices = np.argmax(type_probs_matrix, axis=1)
            predicted_type_probs = np.max(type_probs_matrix, axis=1)
            predicted_types = [label_map.get(idx, 'unknown') for idx in type_predictions_indices]
            
            df_results.loc[indices_fraude, 'predicted_fraud_type'] = predicted_types
            df_results.loc[indices_fraude, 'predicted_type_probability'] = predicted_type_probs
            
            type_probs_list = []
            for probs_row in type_probs_matrix:
                row_dict = {label_map.get(i, 'unknown'): float(prob) for i, prob in enumerate(probs_row)}
                type_probs_list.append(row_dict)
            
            df_results.loc[indices_fraude, 'type_probabilities'] = pd.Series(
                type_probs_list, 
                index=indices_fraude, 
                dtype='object'
            )
        except Exception as e:
            print(f"Erro na predição multiclasse: {e}")
            # Se falhar, o tipo de fraude ficará como 'None'

    # --- LÓGICA DA STRING DE SAÍDA ---
    
    # 1. Definir o padrão (Não é fraude)
    df_results['resumo_da_predicao'] = "Não é uma fraude."
    
    # 2. Atualizar apenas as linhas que SÃO fraude (usando os índices)
    if not indices_fraude.empty:
        tipos_de_fraude = df_results.loc[indices_fraude, 'predicted_fraud_type']
        df_results.loc[indices_fraude, 'resumo_da_predicao'] = "É UMA FRAUDE do tipo " + tipos_de_fraude.astype(str)
        df_results['resumo_da_predicao'] = df_results['resumo_da_predicao'].str.replace("do tipo None", "do tipo desconhecido")

    return df_results

# --- Exemplo de Uso (Simulação) ---
print("\n--- SIMULANDO EXECUÇÃO DE INFERÊNCIA AUTÔNOMA ---")

# 1. Carregar os artefatos UMA VEZ
p_bin, p_multi, l_map = carregar_artefatos(MODEL_DIR)

print("\n--- CÉLULA 10: TESTANDO PAYLOAD DE ATAQUE REALISTA ---")

if 'p_bin' in locals() and p_bin:
    
    # Payload 1 atualizado com as 7 novas features
    transacao_legitima = {
        'valor_transacao': 50.20,
        'data_transacao': datetime.now(),
        'tipo_iniciacao_pix_id': 1, 'finalidade_pix_id': 1,
        'pagador_saldo': 1500.00, 'pagador_conta_aberta_em': '2018-05-10T10:00:00',
        'pagador_tipo_conta_id': 1, 'pagador_natureza_id': 1, 'pagador_data_nascimento': '1985-01-15T00:00:00',
        'recebedor_saldo': 3000.00, 'recebedor_conta_aberta_em': '2019-11-20T14:30:00',
        'recebedor_tipo_conta_id': 1, 'recebedor_natureza_id': 1, 'recebedor_data_nascimento': '1990-03-22T00:00:00',
        'pagador_txs_ultimas_24h': 2, 'pagador_valor_ultimas_24h': 100.0,
        'recebedor_txs_ultima_1h': 0, 'recebedor_valor_ultima_1h': 0.0,
        'pagador_segundos_desde_ultima_tx': 86400,
        # --- Novas Features (Valores legítimos) ---
        'primeira_interacao': 0,
        'pagador_interacoes_com_recebedor': 5,
        'recebedor_num_pagadores_unicos_24h': 10,
        'recebedor_idade_conta_dias': 1500,
        'pagador_idade_conta_dias': 2000,
        'valor_vs_media_pagador_30d': 0.8,
        'valor_vs_saldo_pagador': 0.03 # (50.20 / 1500.00)
    }

    # Payload 2 atualizado com as 7 novas features
    transacao_ataque_fan_in = {
        'valor_transacao': 30.00, 
        'data_transacao': datetime.now().replace(hour=3, minute=30), # Madrugada
        'tipo_iniciacao_pix_id': 2, 'finalidade_pix_id': 2,
        'pagador_saldo': 10.00, # Saldo baixo
        'pagador_conta_aberta_em': datetime.now().replace(day=1), # Conta Nova
        'pagador_tipo_conta_id': 1, 'pagador_natureza_id': 1, 'pagador_data_nascimento': '1998-07-10T00:00:00',
        'recebedor_saldo': 50000.00, 
        'recebedor_conta_aberta_em': datetime.now().replace(day=2), # Conta Nova
        'recebedor_tipo_conta_id': 2, 'recebedor_natureza_id': 2, 'recebedor_data_nascimento': '1999-12-01T00:00:00',
        'pagador_txs_ultimas_24h': 1, 
        'pagador_valor_ultimas_24h': 0.0,
        'recebedor_txs_ultima_1h': 50, # SINAL: 50 txs na última hora
        'recebedor_valor_ultima_1h': 50000.00, # SINAL: R$ 50k recebidos
        'pagador_segundos_desde_ultima_tx': 1,
        # --- Novas Features (Valores de ataque) ---
        'primeira_interacao': 1, # SINAL
        'pagador_interacoes_com_recebedor': 0, # SINAL
        'recebedor_num_pagadores_unicos_24h': 120, # SINAL
        'recebedor_idade_conta_dias': 1, # SINAL
        'pagador_idade_conta_dias': 2, # SINAL
        'valor_vs_media_pagador_30d': None, # SINAL (Imputer vai tratar)
        'valor_vs_saldo_pagador': 3.0 # SINAL (30.00 / 10.00)
    }
    
    payload_realista = pd.DataFrame([transacao_legitima, transacao_ataque_fan_in])
    
    print("Executando predição no payload de ataque realista...")
    
    resultados_realistas = prever_fraude(payload_realista, p_bin, p_multi, l_map)
    
    print("\nResultados da Predição Realista (Saída):")
    output_columns = ['resumo_da_predicao', 'fraud_probability', 'predicted_type_probability']
    print(resultados_realistas[output_columns])

else:
    print("Erro: Modelos não foram carregados. Execute o carregamento primeiro.")